### 1. Download and Load model from S3

In [1]:
# Download model 
import numpy as np 
import tensorflow as tf
import boto3
from tensorflow.keras.models import load_model

s3 = boto3.client('s3')
bucket_name = 'hcp-24021b1a-cd5d-415f-a415-b76fdbc2f686'
object_name = 'data/archive/fruits-360_dataset/fruits-360/model/e4efbee18a28f368/fruitmodel/model.h5'
file_name = 'downloaded_model.h5'
    
s3.download_file(bucket_name, object_name, file_name)
loaded_model = load_model(file_name)


2022-09-18 21:25:08.182253: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-18 21:25:15.320618: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 96, 96, 16)        80        
                                                                 
 activation (Activation)     (None, 96, 96, 16)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        2080      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 24, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 64)        8

In [3]:
#Load model 
#Integer only
#Creating integer only models is a common use case for TensorFlow Lite for Microcontrollers.

loaded_model = tf.keras.models.load_model('/Users/i551982/Desktop/Github/ESP32-tutorial/downloaded_model.h5')



### 2) Convert to Tensorflow model 

In [4]:
WIDTH = 96
HEIGHT = 96

def representative_dataset():
    for _ in range(100):
      data = np.random.rand(1, WIDTH, HEIGHT, 1)
      yield [data.astype(np.float32)]
      
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE] #[tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# converter.inference_input_type = tf.int8  # or tf.uint8
# converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()

file = open( '/Users/i551982/Desktop/Github/ESP32-tutorial/converted_model.tflite' , 'wb' ) 
file.write( tflite_quant_model ) 

INFO:tensorflow:Assets written to: /var/folders/3v/kt3p4yx106n_9h7p5_4lqk7c0000gn/T/tmp6it86o0a/assets


INFO:tensorflow:Assets written to: /var/folders/3v/kt3p4yx106n_9h7p5_4lqk7c0000gn/T/tmp6it86o0a/assets
/Users/i551982/opt/anaconda3/envs/tf/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2022-09-18 21:25:46.291797: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-09-18 21:25:46.291815: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-09-18 21:25:46.292534: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/3v/kt3p4yx106n_9h7p5_4lqk7c0000gn/T/tmp6it86o0a
2022-09-18 21:25:46.295707: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-09-18 21:25:46.295732: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug in

64456

In [5]:
#Check input and output format of quantized model 
interpreter = tf.lite.Interpreter(model_content=tflite_quant_model)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.float32'>
output:  <class 'numpy.float32'>


In [31]:

#https://www.tensorflow.org/lite/performance/post_training_integer_quant
#ESP-EYE Image 
#Apple 

import glob

# image_path = '/Users/i551982/Desktop/Github/ESP32-tutorial/dataset/apple_0.jpg'
# image_path = '/Users/i551982/Desktop/Github/ESP32-tutorial/dataset/apple_1.jpg'
# image_path = '/Users/i551982/Desktop/Github/ESP32-tutorial/dataset/apple_2.jpg'
# image_path = '/Users/i551982/Desktop/Github/ESP32-tutorial/dataset/lemon_0.jpg'
image_path = '/Users/i551982/Desktop/Github/ESP32-tutorial/dataset/lemon_1.jpg'


img_width =  96
img_height = 96

# 1. Read image
img = tf.io.read_file(image_path)

# 2. Decode and convert 
img = tf.io.decode_jpeg(img, channels=1)
# print("Before convert type")
# print(img)
# 3. Convert to float32 in [0, 1] range
img = tf.image.convert_image_dtype(img, tf.float32) 
# img = tf.image.convert_image_dtype(img, tf.int8) 
# print("After")
# print(img)


# # 4. Resize to the desired size
img = tf.image.resize(img, [img_height, img_width], method='nearest') 
# print("Resize")
# print(img)

# interpreter = tf.lite.Interpreter(model_path="/Users/i551982/Desktop/esp-eye/kaist-tutorial-1/converted_model_v20_1.tflite") #Sanraj's model from AI Core
interpreter = tf.lite.Interpreter(model_path="/Users/i551982/Desktop/esp-eye/kaist-tutorial-1/converted_model.tflite") #Sanraj's model from AI Core

interpreter.allocate_tensors()
# print('interpreter', interpreter)

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.expand_dims(img, 0)
# print('expand: ',img)

interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

#print(interpreter.get_tensor(input_details[0]['index']))
# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
preds = interpreter.get_tensor(output_details[0]['index'])
# print("Preds:",preds.shape)
# print(i,":", preds)

# apple_preds = np.append(apple_preds, preds)
# lemon_preds = np.append(lemon_preds, preds)

# print(preds.shape)
# print(type(preds))
pred_texts = np.argmax(preds)

if pred_texts == 0: 
    pred_texts = "Apple"

else: 
    pred_texts = "Lemon"
    
    
print(pred_texts)


34 : [[0.10546875 0.89453125]]
Lemon


### 3) Convert to C++ 


We can then run this command to convert the model to c code.
```
xxd -i converted_model.tflite > firmware/src/model_data.cc
```